## An Automated Portfolio Trading System with Feature Preprocessing and Recurrent Reinforcement Learning

### Setup

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from lib.data import Data, get_buy_and_hold_df, INDICATORS

from lib.model import RRL, train, validation

from lib.backtest import (
    run_rrl_strategy, 
    create_backtest_dataset, 
    generate_layout, 
    generate_nrows, 
    generate_subplot_titles, 
    init_subplots, 
    populate_subplots,
    plot_cumulative_profits, 
    generate_df_barplot, 
    make_cumrets_barplot,
    plot_avg_portfolio_allocation, 
)

from lib.metrics import (
    calc_cumulative_profits, 
    calc_sharpe_ratio, 
)

from lib import save_to_yaml, read_yaml

In [27]:
import numpy as np 
import pandas as pd

from pickle import load, dump

### Data

#### Settings

**Assets**

|  Ticker | Company  |
|---|---|
| XOM | Exxon Mobil Corporation |
| VZ | Verizon Communications Inc. |
| NKE | Nike, Inc. |
| AMAT | Applied Materials, Inc. |
| MCD | McDonald's Corporation |
| MSFT | Microsoft Corporation |
| AAP | Advance Auto Parts, Inc. |
| NOV | Nov, Inc. |

In [75]:
# assets in the portfolio
ASSETS = ["XOM", "VZ", "NKE", "AMAT", "MCD", "MSFT", "AAP", "NOV"]
n_assets = 8
assets = np.random.choice(ASSETS, size=n_assets, replace=False).tolist()
# assets = ["AMAT", "MSFT", "XOM", "NOV"]

# date settings
start_date = "2009-12-31"
end_date = "2017-12-29"
window_size = 100

# technical indicators
momentum = [
    "MOM",
    "MACD",
    "MFI",
    "RSI"
]

volatility = [
    "ATR",
    "NATR"
]

cycle = [ 
    # "HTDCP",
    # "HTS",
    # "HTTMMM"      
]
# nb: cycle indicators seem to decrease the performance of the trading system

volume = [
    "CO",
    "OBV"
]

indicators = momentum + volatility + cycle + volume
indicators

# number of principal components (None means no PCA)
pca_ncp = None

# boolean indicating whether to apply discrete wavelet transformation
dwt = True

#### Instantiate `Data` object

In [76]:
data = Data(start_date, end_date, window_size, assets, indicators, pca_ncp, dwt)
data

[*********************100%***********************]  8 of 8 completed


Data(start_date='2009-12-31', end_date='2017-12-29', window_size=100, assets=['VZ', 'MCD', 'AMAT', 'MSFT', 'NKE', 'AAP', 'NOV', 'XOM'], indicators=['MOM', 'MACD', 'MFI', 'RSI', 'ATR', 'NATR', 'CO', 'OBV'], pca_ncp=None, discrete_wavelet=True)

In [30]:
# save data object into yaml configuration file 

# save_to_yaml(obj=data, path="./config/data.yaml")

### `RRL` - Training and validation

In [77]:
initial_invest = 100
n_epochs = 100

#### No transaction fees 

$\delta=0$

In [78]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features) 
print(rrl) 

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_no_fees_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  88%|████████▊ | 88/100 [00:03<00:00, 28.60it/s, sharpe_ratio=0.0322]  


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 3.871628502389674
Cumulative profits: 118.25408114366223
Training window: 2010-06-30/2010-10-14


Training in progress...:  11%|█         | 11/100 [00:00<00:03, 23.21it/s, sharpe_ratio=0.34]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 3.8967351641377004
Cumulative profits: 138.78810875981003
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -0.41511277907092337
Cumulative profits: 294.52683490352115
Training window: 2017-03-31/2017-07-18


Training in progress...: 100%|██████████| 100/100 [00:03<00:00, 29.76it/s, sharpe_ratio=-.0132]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 5.485406215348063
Cumulative profits: 332.55052664009514
Training window: 2017-08-23/2017-12-07


Training in progress...:   6%|▌         | 6/100 [00:00<00:03, 30.34it/s, sharpe_ratio=0.385]


Less than 30 observations in the validation set after preprocessing.


#### Transactions fees 

$\delta \in \{0.001, 0.003, 0.005\}$

In [79]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.001) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_10bps_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.001, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  36%|███▌      | 36/100 [00:01<00:02, 21.64it/s, sharpe_ratio=-.00492]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 3.2217066915621886
Cumulative profits: 114.60167469501185
Training window: 2010-06-30/2010-10-14


Training in progress...: 100%|██████████| 100/100 [00:04<00:00, 24.73it/s, sharpe_ratio=0.229]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 3.2139019791415273
Cumulative profits: 131.88081276468986
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: -1.1244658734295594
Cumulative profits: 256.7066606051187
Training window: 2017-03-31/2017-07-18


Training in progress...: 100%|██████████| 100/100 [00:03<00:00, 30.24it/s, sharpe_ratio=-.0254]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 3.387469239457244
Cumulative profits: 282.4817758379988
Training window: 2017-08-23/2017-12-07


Training in progress...: 100%|██████████| 100/100 [00:03<00:00, 27.99it/s, sharpe_ratio=0.19]


Less than 30 observations in the validation set after preprocessing.


In [80]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.003) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_30bps_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.003, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...: 100%|██████████| 100/100 [00:03<00:00, 26.22it/s, sharpe_ratio=-.0217]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.4686027040913125
Cumulative profits: 112.16043736939973
Training window: 2010-06-30/2010-10-14


Training in progress...:  21%|██        | 21/100 [00:00<00:02, 28.11it/s, sharpe_ratio=0.107]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 3.104122315321081
Cumulative profits: 127.1879292409818
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: 0.7662494171109729
Cumulative profits: 249.8952011068649
Training window: 2017-03-31/2017-07-18


Training in progress...:   1%|          | 1/100 [00:00<00:05, 16.70it/s, sharpe_ratio=0.0924]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.594401182822278
Cumulative profits: 276.9331453022875
Training window: 2017-08-23/2017-12-07


Training in progress...:  11%|█         | 11/100 [00:00<00:04, 18.76it/s, sharpe_ratio=0.256]


Less than 30 observations in the validation set after preprocessing.


In [81]:
# n_features = data.pca_ncp
n_features = data.n_features

rrl = RRL(n_assets=data.n_assets, n_features=n_features, delta=.005) 
print(rrl)

results = run_rrl_strategy(rrl, data, n_epochs, initial_invest)

file_path = "./backup/rrl_50bps_dwt_1.pkl"

with open(file_path, "wb") as f: dump(results, f)

RRL(n_assets=8, n_features=8, delta=0.005, rho=0.1, l2=0.01)
Training window: 2010-02-05/2010-05-24


Training in progress...:  22%|██▏       | 22/100 [00:00<00:03, 24.86it/s, sharpe_ratio=-.0767]


Validation window: 2010-06-30/2010-10-14
Sharpe ratio on validation set: 2.9916777566143784
Cumulative profits: 112.90889225154841
Training window: 2010-06-30/2010-10-14


Training in progress...:  24%|██▍       | 24/100 [00:00<00:02, 31.19it/s, sharpe_ratio=0.239]


Validation window: 2010-11-19/2011-03-09
Sharpe ratio on validation set: 3.0955369154773327
Cumulative profits: 127.95388545267608
Training window: 2010-11-19/2011-03-09


...

Validation window: 2017-03-31/2017-07-18
Sharpe ratio on validation set: 1.0264643201587662
Cumulative profits: 231.19990516204555
Training window: 2017-03-31/2017-07-18


Training in progress...:  28%|██▊       | 28/100 [00:01<00:02, 25.09it/s, sharpe_ratio=-.0367]


Validation window: 2017-08-23/2017-12-07
Sharpe ratio on validation set: 2.5720551705843158
Cumulative profits: 251.16506396695303
Training window: 2017-08-23/2017-12-07


Training in progress...:   9%|▉         | 9/100 [00:00<00:03, 29.42it/s, sharpe_ratio=0.242]


Less than 30 observations in the validation set after preprocessing.


### Backtest

In [ ]:
# define initial invested amount
initial_invest = 100

# compute cumulative returns for "buy & hold" strategy
buy_and_hold = get_buy_and_hold_df(data.df)
buy_and_hold_sr = calc_sharpe_ratio(returns=buy_and_hold.returns, window_size=252)

#### RRL-PCA-DWT : Different transaction fees

In [97]:
# enter a version to backtest ("", "pca", "dwt", "pca_dwt")
version = "pca_dwt"

file_names = [f"rrl_{fees}_{version}" for fees in ("no_fees", "10bps", "30bps", "50bps")]

path = "./backup/"
results = {
    f.replace("results_", ""): load(open(f"{path}{f}_1.pkl", "rb"))
    for f in file_names
}

keys = [
    " ".join(key.split("_")[:-1])
    for key in results.keys()
]
values = list(results.values())

results = dict(zip(keys, values))

In [55]:
dfs_backtest = {k: create_backtest_dataset(v) for k, v in results.items()}

In [58]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold, 
    buy_and_hold_sr, 
    initial_invest, 
    layout)

fig.show()

In [59]:
fig.write_image("imgs/cum-profits-rrl-pca-dwt-1.png")

### Analysis per trading window for one setting

In [122]:
version = "dwt" 
fees = "30bps"
setting = f"rrl_{fees}_{version}"

results = load(open(f"{path}{setting}_1.pkl", "rb"))

df = create_backtest_dataset(results)
batch_names = np.unique(df["Batch"])

In [126]:
metric ="Cumulative returns"
setting = " ".join(setting.split("_"))

fig_title = f"{metric} per trading window | {setting}"

ncols = 4
nrows = generate_nrows(len(batch_names), ncols)

titles = generate_subplot_titles(batch_names)

fig = init_subplots(nrows, ncols, titles)
fig = populate_subplots(fig, df, batch_names, titles, ncols, y_varname=metric, initial_invest=initial_invest)

layout = generate_layout(
    fig_height=1200, 
    fig_width=1400, 
    fig_title=fig_title)
    
fig.update_layout(layout)

In [136]:
df_barplot = generate_df_barplot(df)
title = f"% Cumulative returns at the end of each trading window | {setting}"

layout = generate_layout(
    fig_height=500, 
    fig_width=900, 
    fig_title=title,
    x_axis_args={"title": {"text": "Trading window"}}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "%"}}
)
fig = make_cumrets_barplot(df_barplot, layout)

fig.show()

In [137]:
# fig.write_image("imgs/cum-rets-rrl-dwt-30bps.png")

**Portfolio allocation**

In [138]:
layout = generate_layout(
    fig_height=500, 
    fig_width=700, 
    fig_title="Average portfolio allocation over trading windows"
)

fig = plot_avg_portfolio_allocation(results, layout)

fig.show()

In [139]:
# fig.write_image("imgs/port-allocation-rrl-dwt-30bps.png")

#### Comparison between different preprocessing layers

| PCA | DWT | Preprocessing layer |
|---|---|---|
| 1 | 1 |Full|
|1 | 0 |Partial|
| 0 | 1 |Partial|
| 0 | 0 |None|

In [114]:
# enter a level of transaction fees ("no_fees", "10bps", "30bps", "50bps")
fees = "30bps"

file_names = [f"rrl_{fees}{version}" for version in ("", "_dwt", "_pca", "_pca_dwt")]

path = "./backup/"
results = {
    f: load(open(f"{path}{f}_1.pkl", "rb"))
    for f in file_names
}

keys = [
    " ".join(key.split("_"))
    for key in results.keys()
]
values = list(results.values())

results = dict(zip(keys, values))

In [115]:
dfs_backtest = {k: create_backtest_dataset(v) for k, v in results.items()}

In [116]:
layout = generate_layout(
    fig_height=500, 
    fig_width=1100, 
    fig_title="", 
    x_axis_args={"gridcolor": "lightgrey"}, 
    y_axis_args={"gridcolor": "lightgrey", "title": {"text": "$"}}
)
fig = plot_cumulative_profits(
    dfs_backtest, 
    buy_and_hold, 
    buy_and_hold_sr, 
    initial_invest, 
    layout)

fig.show()

In [117]:
fig.write_image("imgs/cum-profits-rrl-30bps.png")